In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Đổi đường dẫn này thành đường dẫn đến thư mục r(2+1)D trong drive của chú
%cd /content/drive/MyDrive/Anh_Qui
%ls

/content/drive/MyDrive/Anh_Qui
data_train.csv  data_val.csv  label.txt  outputs/  Train/  Val/


In [ ]:
%pip install torch-summary

In [ ]:
import os
import cv2
import time

import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import albumentations as A
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary


# Thay đổi giá trị default của các argument, hạn chế thay đổi code bên dưới
ap = argparse.ArgumentParser()
ap.add_argument('-f')
ap.add_argument('--model_file', default='outputs/checkpoints/model_TSNsss.pth',
                help='path to save the trained model.')
ap.add_argument('--data_dir_train_0', default='Train/0',
                help='path to data folder.')
ap.add_argument('--data_dir_train_1', default='Train/1',
                help='path to data folder.')
ap.add_argument('--data_dir_val_0', default='Val/0',
                help='path to data folder.')
ap.add_argument('--data_dir_val_1', default='Val/1',
                help='path to data folder.')
ap.add_argument('--data_file_train', default='data_train.csv',
                 help='path to data file.')
ap.add_argument('--data_file_val', default='data_val.csv',
                help='path to data file.')
ap.add_argument('--label_file', default='label.txt',
                help='path to label file.')
ap.add_argument('--output_dir', default='outputs',
                help='path to output folder.')

ap.add_argument('--test_size', type=float, default=0.2,
                help='percentage of test set.')
ap.add_argument('--epochs', type=int, default=10,
                help='number of epochs to train our network for.')
ap.add_argument('--batch_size', type=int, default=32)
ap.add_argument('--lr', type=float, default=1e-3)
ap.add_argument('--save_frequency', type=int, default=1,
                help='number of epochs between each checkpoint.')
args = ap.parse_args()

# check cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

# get clip and label
# df = pd.read_csv(args.data_file)
# clips = df['clip'].tolist()

# # change string type to array type
# def alter_type(clip):
#     clip = clip.replace("'", "").replace("[", "").replace("]", "")
#     clip = clip.replace("/content/drive/MyDrive/Colab Notebooks/frames_temp", args.data_dir)
#     clip = clip.split(", ")
#     return clip

# # process all clip_len frames to get X
# def get_clip_input(clips):
#     return [alter_type(clip) for clip in clips]
def load_clip_data(data_dir, data_file):
    # get clip and label
    df = pd.read_csv(data_file)
    clips = df['clip'].tolist()

    # change string type to array type
    def alter_type(clip):
        clip = clip.replace("'", "").replace("[", "").replace("]", "")
        clip = clip.replace("/content/drive/MyDrive/Colab Notebooks/frames_temp", data_dir)
        clip = clip.split(", ")
        return clip

    # process all clip_len frames to get X
    def get_clip_input(clips):
        return [alter_type(clip) for clip in clips]
    
    X = get_clip_input(clips)
    y = df['label'].tolist()
    return X, y

with open(args.label_file, 'r') as f:
    class_names = [_.strip() for _ in f.readlines()]

# # get X, y
# X = get_clip_input(clips)
# y = df['label'].tolist()
# print('X:', len(X))
# print('y:', len(y))

X_RGB, y = load_clip_data(args.rgb_data_dir, args.rgb_data_file)
X_OF, y = load_clip_data(args.of_data_dir, args.of_data_file)
print('Xs:', len(X_RGB), len(X_OF))
print('y:', len(y))

# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=args.test_size, random_state=42)
print(f'Training instances: {len(X_train)}')
print(f'Validataion instances: {len(X_test)}')
print()

# custom dataset
class UCF11(Dataset):
    def __init__(self, clips, targets, transform=None, dtype=torch.float32, device='cpu'):
        self.clips = clips
        self.targets = targets
        self.labels = np.unique(self.targets).tolist()
        self.target2id = lambda _: self.labels.index(_)
        self.y = torch.tensor([self.target2id(_) for _ in self.targets],
                              dtype=torch.long)
        self.transform = transform
        self.dtype = dtype
        self.device = device

    def __len__(self):
        return len(self.clips)

    def __getitem__(self, i):
        clip = self.clips[i]
        input_frames = []
        for frame in clip:
            image = Image.open(frame)
            image = image.convert('RGB')
            image = np.array(image)
            if self.transform is not None:
                image = self.transform(image=image)['image']
            input_frames.append(image)
        input_frames = np.asarray(input_frames)
        # print('input_frames.shape: ', input_frames.shape)
        # input_frames = np.expand_dims(input_frames, axis=0)
        input_frames = np.transpose(input_frames, (3, 0, 1, 2))
        input_frames = torch.tensor(input_frames,
                                    dtype=self.dtype,
                                    device=self.device)
        # label
        y = self.y[i].to(self.device)
        return (input_frames, y)

transform = A.Compose([
    A.Resize(128, 171, always_apply=True),
    A.CenterCrop(112, 112, always_apply=True),
    A.Normalize(mean=[0.43216, 0.394666, 0.37645],
                std=[0.22803, 0.22145, 0.216989],
                always_apply=True)
])

train_set = UCF11(X_train, y_train, transform=transform, device=device)
val_set = UCF11(X_test, y_test, transform=transform, device=device)

train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=args.batch_size, shuffle=True)

# model
model = torchvision.models.video.r2plus1d_18(pretrained=True, progress=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)
# summary(model, (3, 16, 112, 112))
summary(model, (3, 8, 112, 112))
print()

# criterion
criterion = nn.CrossEntropyLoss()

# optim
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)
# scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=True
)

# training
def fit(model, train_loader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (X, y) in pbar:
        optimizer.zero_grad()
        outputs = model(X)
        preds = outputs.argmax(1)

        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        train_running_loss += loss.item()
        train_running_correct += (preds == y).sum().item()
        pbar.set_description(f'[Training iter {i+1}/{len(train_loader)}] batch_loss={loss.item():.03f}')
    train_loss = train_running_loss / len(train_loader.dataset)
    train_accuracy = 100. * train_running_correct / len(train_loader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    return train_loss, train_accuracy

# validating
@torch.no_grad()
def validate(model, test_loader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    pbar = tqdm(enumerate(test_loader), total=len(test_loader))
    for i, (X, y) in pbar:
        outputs = model(X)
        preds = outputs.argmax(1)

        loss = criterion(outputs, y)
        val_running_loss += loss.item()
        val_running_correct += (preds == y).sum().item()

        pbar.set_description(f'[Validation iter {i+1}/{len(train_loader)}] batch_loss={loss.item():.03f}')
    val_loss = val_running_loss / len(test_loader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_loader.dataset)
    print(f'Val loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
    return val_loss, val_accuracy


train_loss, train_accuracy = [], []
val_loss, val_accuracy = [], []
start = time.time()
start_epoch = 0

 # resume from previous states
#  len() to check file_name 
if len(args.model_file) and os.path.exists(args.model_file):
    print(f'Loading checkpoint from {args.model_file}')
    state_dicts = torch.load(args.model_file, map_location=device)
    start_epoch = state_dicts['epoch']
    model.load_state_dict(state_dicts['model_state_dict'])
    optimizer.load_state_dict(state_dicts['optimizer_state_dict'])
    scheduler.load_state_dict(state_dicts['scheduler_state_dict'])

    train_loss = state_dicts['train_loss']
    train_accuracy = state_dicts['train_accuracy']
    val_loss = state_dicts['val_loss']
    val_accuracy = state_dicts['val_accuracy']
    # del state_dicts là để xóa tên biến 
    del state_dicts

for epoch in range(start_epoch, args.epochs):
    print(f"[Epoch {epoch+1} / {args.epochs}]")

    train_epoch_loss, train_epoch_accuracy = fit(model, train_loader)
    val_epoch_loss, val_epoch_accuracy = validate(model, val_loader)

    # print(f'[Epoch {epoch+1} / {args.epochs}] Training summary: train_loss={train_loss:.3f}, train_acc={train_accuracy:.3f}')
    # print(f'[Epoch {epoch+1} / {args.epochs}] Validation summary: val_loss={val_loss:.3f}, val_acc={val_accuracy:.3f}')

    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    scheduler.step(val_epoch_loss)

    # save state dicts after each epoch
    if len(args.model_file) and ((epoch > 0 and epoch % args.save_frequency == 0) or epoch == args.epochs - 1):
        os.makedirs(os.path.dirname(args.model_file), exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
            }, args.model_file)
        print(f'\t[Epoch {epoch+1} / {args.epochs}] Model checkpointed at {args.model_file}')
    print()

end = time.time()
print(f'TRAINING COMPLETED! Total elapsed time: {(end-start)/60:.3f} minutes')

# accuracy plots
plt.figure(figsize=(10, 7))
plt.plot(train_accuracy, color='green', label='train accuracy')
plt.plot(val_accuracy, color='blue', label='validataion accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.savefig(os.path.join(args.output_dir, 'accuracy_train_TSN.png'))

# loss plots
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='orange', label='train loss')
plt.plot(val_loss, color='red', label='validataion loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.legend()
plt.savefig(os.path.join(args.output_dir, 'loss_train_TSN.png'))

# serialize the model to disk
print('Saving model...')
torch.save(model.state_dict(), os.path.join(args.output_dir, 'r2plus1d__TSN_OF.pth'))